In [25]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="darkgrid")

In [26]:
gdp = pd.read_csv('./Data/GDP.csv')


ISO_CODES = ['CHN', 'DEU', 'IND', 'USA', 'WLD']

gdp = gdp[gdp['Country Code'].isin(ISO_CODES)]
gdp = gdp.loc[:,(gdp.columns.isin(['Country Name'])) | (gdp.columns >= '2004')]
gdp = gdp.drop(['Country Code', 'Indicator Name', 'Indicator Code'], axis=1)
gdp = gdp.fillna(0)         
gdp = gdp.rename({'Country Name': 'Country'}, axis=1)

gdp = pd.melt(gdp, id_vars='Country', value_vars=np.arange(2004, 2020).astype(str),
                   var_name='Year', value_name='GDP')


gdp_world = gdp[gdp['Country'] == 'World'].reset_index(drop=True)
gdp = gdp[gdp['Country'] != 'World'].reset_index(drop=True)

gdp_relative = []

for _, r in gdp.iterrows():
    y = r['Year']
    v = r['GDP']
    
    V = gdp_world[gdp_world['Year'] == y]['GDP'].values[0]
    
    gdp_relative.append(v / V)
    
gdp_relative = pd.Series(gdp_relative)
gdp_relative = np.round(gdp_relative, 2)

gdp['GDP Relative'] = gdp_relative

In [56]:
df = pd.read_csv('./Data/investment_trends.csv', index_col=0)

df = df.loc[df.index.repeat(4)]
df = df.reset_index()
df = df.rename({'index': 'Year'}, axis=1)

df.insert(0, 'Country', gdp['Country'])

for i, r in df.iterrows():
    c = r['Country']
    y = str(r['Year'])
    
    v = gdp[(gdp['Country'] == c) & (gdp['Year'] == y)]
    v = v['GDP Relative']
    v = v.values[0]
    
    df.at[i, 2:] = r[2:] * v
    
df

/home/stefan/anaconda3/envs/ds-praktikum/lib/python3.8/site-packages/pandas/core/frame.py:3155: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  self.loc[index, col] = value


,Country,Year,Government R&D,Corporate R&D,Venture capital,Private equity expansion capital,Public markets,Asset finance,Asset Finance minus Re-invested equity,Small distributed capacity,Private equity buy-outs,Public markets investor exits,Corporate M&A,Project acquisition & refinancing,Total: M&A/ buy-outs etc.
0,China,2004,0.076,0.076,0.016,0.012,0.012,1.284,1.280,0.320,0.032,0.016,0.088,0.204,0.340
1,Germany,2004,0.114,0.114,0.024,0.018,0.018,1.926,1.920,0.480,0.048,0.024,0.132,0.306,0.510
2,India,2004,0.038,0.038,0.008,0.006,0.006,0.642,0.640,0.160,0.016,0.008,0.044,0.102,0.170
3,United States,2004,0.532,0.532,0.112,0.084,0.084,8.988,8.960,2.240,0.224,0.112,0.616,1.428,2.380
4,China,2005,0.100,0.095,0.030,0.050,0.180,2.500,2.490,0.505,0.185,0.115,0.380,0.625,1.310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,United States,2018,1.320,1.872,0.048,0.528,1.440,58.080,56.688,9.168,3.120,0.024,3.504,29.712,36.360
60,China,2019,0.912,1.232,0.032,0.288,1.056,36.816,36.272,8.336,0.512,0.000,2.192,13.408,16.112
61,Germany,2019,0.228,0.308,0.008,0.072,0.264,9.204,9.068,2.084,0.128,0.000,0.548,3.352,4.028
62,India,2019,0.171,0.231,0.006,0.054,0.198,6.903,6.801,1.563,0.096,0.000,0.411,2.514,3.021


In [57]:
n = df.columns[2:].size
countries = ['China', 'Germany', 'India', 'United States']

for country in countries:
    for year in np.arange(2000, 2004):
        row = [country, year] + list(np.zeros(n))
        df.loc[len(df), :] = row

In [60]:
df = df.sort_values(by=['Country', 'Year'], ignore_index=True)

In [62]:
df['Year']

0     2000.0
1     2001.0
2     2002.0
3     2003.0
4     2004.0
       ...  
75    2015.0
76    2016.0
77    2017.0
78    2018.0
79    2019.0
Name: Year, Length: 80, dtype: float64

In [51]:
df.to_csv('./Data Formated/investment_trends.csv')